# 1. Objetivo

La idea del proycto es realizar una clasificación de imágenes en tres clases -> Frutas, Vegetales y Paquetes.

#### data set reference
```
  title={A Hierarchical Grocery Store Image Dataset with Visual and Semantic Labels},
  author={Klasson, Marcus and Zhang, Cheng and Kjellstr{\"o}m, Hedvig},
  booktitle={IEEE Winter Conference on Applications of Computer Vision (WACV)},
  year={2019}
```

## Un puntapie de inicio

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from six.moves import urllib
import io
import shutil

from IPython.display import clear_output, Image, display, HTML


import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import time
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from keras.preprocessing import image
from keras import regularizers
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten
from keras import backend as K

#### Vamos a crear algunas funciones útiles

In [4]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    cm = confusion_matrix(y_true, y_pred)
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

  


# 2. Descarga y Armado del dataset

In [5]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset

Cloning into 'GroceryStoreDataset'...
remote: Enumerating objects: 6556, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6556 (delta 0), reused 1 (delta 0), pack-reused 6553
Receiving objects: 100% (6556/6556), 116.25 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (313/313), done.


In [6]:
## fijamos los paths
train_path = './GroceryStoreDataset/dataset/train' 
validation_path = './GroceryStoreDataset/dataset/val'
test_path = './GroceryStoreDataset/dataset/test'

In [7]:
!ls GroceryStoreDataset/dataset/train/Fruit

Apple	 Kiwi	Mango	   Orange	  Peach      Plum	     Satsumas
Avocado  Lemon	Melon	   Papaya	  Pear	     Pomegranate
Banana	 Lime	Nectarine  Passion-Fruit  Pineapple  Red-Grapefruit


## 2.2 Preprocesamiento inicial de los datos¿
Creamos algunos preprocesamientos sin ningun variación ni aumento de los datos. Este es el comienzo :)

In [8]:
train_batches  = ImageDataGenerator().flow_from_directory(
    train_path, target_size=(224,224), classes = ['Fruit', 'Packages', 'Vegetables'], batch_size = 32)

validation_batches  = ImageDataGenerator().flow_from_directory(
    validation_path, target_size=(224,224), classes = ['Fruit', 'Packages', 'Vegetables'], batch_size = 8)

test_batches  = ImageDataGenerator().flow_from_directory(
    test_path, target_size=(224,224), classes = ['Fruit', 'Packages', 'Vegetables'], batch_size = 32)

Found 2640 images belonging to 3 classes.
Found 296 images belonging to 3 classes.
Found 2485 images belonging to 3 classes.


In [9]:
#take a look at output of the generators

for data_batch, labels_batch in train_batches:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
    

data batch shape: (32, 224, 224, 3)
labels batch shape: (32, 3)


# 3. Crear un Modelo base Convolusional

In [10]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(96, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 96)                37850208  
                                                                 
 dense_1 (Dense)             (None, 3)                 291       
                                                                 
Total params: 37,851,395
Trainable params: 37,851,395
Non-trainable params: 0
____________________________________________

In [11]:
train_filenames = train_batches.filenames
steps_train = len(train_filenames)/train_batches.batch_size

validation_filenames = validation_batches.filenames
steps_valid = len(validation_filenames)/validation_batches.batch_size

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
fit_generator1 = model.fit(
      train_batches,
      steps_per_epoch=steps_train,
      epochs=5,
      validation_data=validation_batches,
      validation_steps=steps_valid)

AttributeError: ignored

In [12]:

import matplotlib.pyplot as plt

acc = fit_generator1.history['acc']
val_acc = fit_generator1.history['val_acc']
loss = fit_generator1.history['loss']
val_loss = fit_generator1.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc', color = 'r')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss', color = 'r')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: ignored

### 3.1.1 Experimentar con el modelo... Prueben hacer un modelo un poco más complejo, con:
 
 Capa convolusional con 32 neuronas
 
 Capa Pooling
 
 Capa Convolusional con 64 neuronas
 
 Capa Pooling
 
 Capa Convolusional con 128 neuronas
 
 Capa Pooling
 
 Capa Convolusional con 128 neuronas
 
 Capa Pooling
 
 Capa de Aplanamiento
 
 Capa Densa con 512 neuronas
 
 Capa de clasificación con la categorías.
 
 ** y obviamente probar con más épocas ** 
 
 Se recomienda entrenar con gpu (o en colab), aunque puede andar sin, aunque bastante lento

### 3.2 Hacer Aumento de datos

Para el mismo modelo que armamos arriba (es decir, una vez que definan uno, se quedan con ese), hagamos un poco de aumento de cantidad de datos. Es una buena forma de tener más datos y prevenir también el overfitting. 

Se puede aumentar los datos con los generadores de entrenamiento. Se recomienta usar reescalamiento, recorte, zoom y espejado horizontal.

* reescalar los inputs de 0,255 a 0,1
* aplicar el rango de recorte (shear_range) para aplicar cortes aleatorios
* aplicar rango de zoom (zoom_range) para aplicar zoom aleatorio a las imágenes
* poner true el espejado horizontal (horizontal_flip) para obtener imágenes espejo horizontales)

Recuerden usar las mismas funciones de activación y optimización, para poder probar el mismo modelo.

Siempre recuerden plotear validación contra training para ver si hay overfitting.

Recuerden que hay que reescalar el test también, porque el modelo aprendió a utilizar los datos escalados

un buen lugar para consultar sobre Image Data Generator (la herramienta para aumentar los datos)
https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/

In [13]:
#### Les dejo algo de código para que vayan empezando

train_datagen = ImageDataGenerator(
        rotation_range=###,
        width_shift_range=###,
        height_shift_range=0.###,
        fill_mode=#####,
        rescale=####,
        shear_range=####,
        zoom_range=####,
        horizontal_flip=####)

test_datagen = ImageDataGenerator(rescale=####)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=50,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_path,
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

train_filenames = train_generator.filenames
steps_train = len(train_filenames)/train_generator.batch_size

validation_filenames = validation_generator.filenames
steps_valid = len(validation_filenames)/validation_generator.batch_size

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

fit_generator_2 = model.fit(
        train_generator,
        steps_per_epoch=steps_train,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=steps_valid)




SyntaxError: ignored

### 3.3 Más aumentos de imágenes 

Probemos agregar otros aumentos:

* rango de rotación (rotation_range) rota las imagenes.
* desplazar las imagenes aleatoriamente con (width_shift) en ancho
* desplazar las imagenes aleatoriamente con (height_shift) en ancho
* poner fill_mode en nearest para completar la imagen.


### 3.3 Regularización: Añadir capa o capas de dropout para regularizar.


# 4. Utilizar una Red Preentrenada y hacer Fine-Tuning!

Probar con VGG16, recortando las últimas capas de la red para hacer fine tuning. El que se anime, puede probar ResNet también que es una red con una arquitectura bastante más compleja!

Recuerden que tienen que setear cuáles serán las capas a entrenar.
Les dejo documentación al respecto! :)

https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/

In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from six.moves import urllib
import io
import shutil
import keras
from IPython.display import clear_output, Image, display, HTML


import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import time
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing import image
from keras import regularizers
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten
from keras import backend as K

In [ ]:
#### Recuerden que aquí, cuando ya estén medio estables con los resultados, pueden empezar a jugar con los optimizadores y también realizar el aumento de imágenes para el modelo con vgg o resnet :)
#### SUERTE Y CUALQUIER COSA  ME PREGUNTAN!